###### ДЗ 2 Кластеризация 

*Креативная часть:* Релизовать на питоне ДЗ от вашего преподователя по МО

*Техническая часть:* сделать под это всё удобные обёртки. Например, это функция, которая просто принимает список фичей, а на выходе показывает результат фита

In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model, model_selection
from sklearn.preprocessing import OrdinalEncoder
import pyclustering as pcl
import matplotlib.pyplot as plt

### Просмотрим и подготовим данные

##### Data fields
**name**: this is the name for each player\
**wins** : number of times the player win a match\
**kills** : number of kills the player made in all his matches\
**kdRatio** : kill/deaths ratio that means, if a player has 10 kills and 5 deaths, his KD ratio is equal to 2. A KD ratio of 1 means that the player got killed exactly as many times as he successfully eliminated his opponents\
**killstreak** : kill a number of enemy players without dying.\
**level** : is the player grade\
**losses** : total number of losing\
**prestige**: it is an optional Mode that players can choose after they progress to Level 55 and max\
**hits** : number of times the player damaged another player\
**timePlayed** : the time spent by every player playing Call of Duty in hours\
**headshots** : number of times the player hit the others with headshots\
**averageTime** : avrage time\
**gamesPlayed** : number of times the player play multiplyer match\
**assists** : number of times player damaging an enemy but a teammate gets the kill.\
**misses** : the number of times the player miss the hit\
**xp** : Experience Points (XP) are a numerical quantity exclusive to multiplayer that dictates a player's level and progress in the game.\
**scorePerMinute** :a measure of how many points players are gaining per unit time.\
**shots** : number of shots the player did\
**deaths** : number of time the player gots killed in the game.

1. Пропущенных данных в датасете нет, можно считывать

In [3]:
df_cod = pd.read_csv("./cod.csv")
df_cod

,name,wins,kills,kdRatio,killstreak,level,losses,prestige,hits,timePlayed,headshots,averageTime,gamesPlayed,assists,misses,xp,scorePerMinute,shots,deaths
0,RggRt45#4697369,0,0,0.000000,0,1,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0
1,JohniceRex#9176033,0,0,0.000000,0,1,0,110,0,7,0,7.0,0,0,0,700,0.0,0,16
2,bootybootykill#1892064,0,66,1.031250,0,9,0,110,0,32,16,32.0,0,1,0,48300,0.0,0,64
3,JNaCo#5244172,3,2,0.400000,0,1,0,0,0,3,0,3.0,0,0,0,1150,0.0,0,5
4,gomezyayo_007#6596687,0,2,0.200000,0,1,0,110,0,5,1,5.0,0,0,0,1000,0.0,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1553,ImranePROPG#2085215,0,51,0.432203,0,10,0,110,0,71,11,71.0,0,0,0,49935,0.0,0,118
1554,Miguel_mor#1492856,0,0,0.000000,0,1,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0
1555,Trianthor#3462590,0,0,0.000000,0,1,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0
1556,tinytrex594#3976133,0,0,0.000000,0,1,0,0,2,1,0,1.0,0,1,7,0,91.2,9,0


2. Проверим данные на непротиворечивость.

Столбцы wins (победы в матчах), losses (проигрыши в матчах) и gamesPlayed (сыгранные матчи) имеют прямую зависимость: сумма побед и поражений должна быть ниже, чем общее количество матчей. Данное условие не выполняется, это можно заключить даже при первом взгляде на данные (строки 3 и 1557), проверим много ли противоречащих данных.

In [6]:
df_cod[(df_cod['gamesPlayed'] < (df_cod['wins']+df_cod['losses']))]

,name,wins,kills,kdRatio,killstreak,level,losses,prestige,hits,timePlayed,headshots,averageTime,gamesPlayed,assists,misses,xp,scorePerMinute,shots,deaths
3,JNaCo#5244172,3,2,0.400000,0,1,0,0,0,3,0,3.000000,0,0,0,1150,0.000000,0,5
5,Brxndoon7-LK#4002715,684,27011,1.066743,18,177,10,110,98332,1366,5113,2.323129,588,6063,305319,3932335,255.672035,403651,25321
6,bdooory_ab#7095171,4,162,0.632812,4,6,2,0,568,8,35,2.000000,4,68,4836,24485,265.500000,5404,256
7,ahevepluto#3505304,186,1898,0.569628,13,37,7,2,5111,550,485,3.666667,150,488,39978,458269,180.379636,45089,3332
9,RPDUNKduo#8122914,26,349,0.444020,7,12,4,0,996,44,40,2.933333,15,138,4844,72765,253.677273,5840,786
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,BlacKPoisoN#9748555,95,3230,0.911400,10,74,7,111,7467,507,702,5.571429,91,1184,40347,1169188,104.544379,47814,3544
1546,Lekestrell#9382004,37,1018,1.014955,11,22,6,24,3481,92,162,2.705882,34,212,14816,178755,203.210870,18297,1003
1549,MeaslyPanther81#4102876,23,406,0.821862,12,10,6,1,1946,65,132,2.321429,28,119,7159,57364,187.126154,9105,494
1550,bimo09#9025166,408,7530,0.799873,12,77,8,28,33533,694,1237,2.679537,259,1792,157188,1208952,296.200288,190721,9414


Почти весь датасет соткан из противоречий.....................

In [9]:
df_cod[(df_cod['gamesPlayed'] >= (df_cod['wins']+df_cod['losses']))]

,name,wins,kills,kdRatio,killstreak,level,losses,prestige,hits,timePlayed,headshots,averageTime,gamesPlayed,assists,misses,xp,scorePerMinute,shots,deaths
0,RggRt45#4697369,0,0,0.000000,0,1,0,0,0,0,0,0.000000,0,0,0,0,0.000000,0,0
1,JohniceRex#9176033,0,0,0.000000,0,1,0,110,0,7,0,7.000000,0,0,0,700,0.000000,0,16
2,bootybootykill#1892064,0,66,1.031250,0,9,0,110,0,32,16,32.000000,0,1,0,48300,0.000000,0,64
4,gomezyayo_007#6596687,0,2,0.200000,0,1,0,110,0,5,1,5.000000,0,0,0,1000,0.000000,0,10
8,MilkyLemonz-_-#5981249,741,21803,1.036658,26,185,29,111,81361,2442,3894,2.826389,864,4029,327230,4269370,198.844226,408591,21032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1552,LokiiFN#2367402,0,0,0.000000,0,1,0,0,0,1,0,1.000000,0,0,0,200,0.000000,0,2
1553,ImranePROPG#2085215,0,51,0.432203,0,10,0,110,0,71,11,71.000000,0,0,0,49935,0.000000,0,118
1554,Miguel_mor#1492856,0,0,0.000000,0,1,0,0,0,0,0,0.000000,0,0,0,0,0.000000,0,0
1555,Trianthor#3462590,0,0,0.000000,0,1,0,0,0,0,0,0.000000,0,0,0,0,0.000000,0,0
